In [26]:
import boto3

sagemaker = boto3.client('sagemaker')

model_name='spark-regression-model-008'
endpoint_config_name='inference-server-008a-config'
endpoint_name='inference-server-008a'

## Create SageMaker Model
role = 'arn:aws:iam::<account number>:role/sagemaker_role'
primary_container = {
    'Image': '959874710265.dkr.ecr.us-west-2.amazonaws.com/inference-server:latest',
    'ModelDataUrl': 's3://neilawsml2/models/mleap/pipeline-model-1.tgz'
}

create_model_response = sagemaker.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-west-2:959874710265:model/spark-regression-model-008


In [27]:
## Create EndPoint Config
response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'default-variant-name',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge'
        },
    ])

print (response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:959874710265:endpoint-config/inference-server-008a-config', 'ResponseMetadata': {'RequestId': 'd0f4c278-c0c5-4dd3-945f-4951005c4a69', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Wed, 21 Feb 2018 08:30:12 GMT', 'x-amzn-requestid': 'd0f4c278-c0c5-4dd3-945f-4951005c4a69', 'content-length': '109', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [28]:
## Create Sagemaker Endpoint
response = sagemaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print (response)

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:959874710265:endpoint/inference-server-008a', 'ResponseMetadata': {'RequestId': '2a2c0071-d6b3-478c-8eb2-403910a2ca32', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Wed, 21 Feb 2018 08:30:14 GMT', 'x-amzn-requestid': '2a2c0071-d6b3-478c-8eb2-403910a2ca32', 'content-length': '89', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [25]:
import boto3, io, json

client = boto3.client('sagemaker-runtime')

input='{"schema":{"fields":[{"name":"Price","type":"double"},{"name":"Mileage","type":"integer"}, \
{"name":"Make","type":"string"},{"name":"Model","type":"string"},{"name":"Trim","type":"string"}, \
{"name":"Type","type":"string"},{"name":"Cylinder","type":"integer"},{"name":"Liter","type":"double"}, \
{"name":"Doors","type":"integer"},{"name":"Cruise","type":"integer"},{"name":"Sound","type":"integer"}, \
{"name":"Leather","type":"integer"}]}, \
"rows":[[9041.9062544231,26191,"Chevrolet","AVEO","SVM Sedan 4D","Sedan",4,1.6,4,0,0,1]]}'
    
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=input,
    ContentType='application/json',
    Accept='application/json'
)

res_json = json.loads(response['Body'].read().decode("utf-8"))

print (json.dumps(res_json, indent=2))

{
  "schema": {
    "fields": [
      {
        "name": "prediction",
        "type": {
          "type": "basic",
          "base": "double",
          "isNullable": false
        }
      }
    ]
  },
  "rows": [
    [
      10236.175823272792
    ]
  ]
}
